# Feature Engineering (Part II)

Update 9-25-25: I am adjusting this notebook to work with the new KPI files from the GitHub Repository.

This notebook will be used to build the function that will calculate all of the quarterly percentage changes and rate of change features. It will add the data back to the file. Consider calcualtions for percent of revenue.


In [1]:
# File system libraries
import os
#from google.colab import drive

# Data Manipulation Libraries
import numpy as np
import pandas as pd

# Stat Libraries
import scipy.stats as stats

# Machine Learning Libraries
#import pycaret #Not working with this version of python
import sklearn

# Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt


In [2]:
# show decimals without scientific notation
pd.set_option('display.float_format', '{:,.2f}'.format)

In [4]:
# Mount the google drive
# drive.mount('/content/drive')
# Navigate to the folder and set the file name
path = '../../datasets'

os.chdir(path)
os.getcwd()
os.listdir()

['.train_test.split',
 'Russell_3000.csv',
 'Russell_3000_Cleaned.csv',
 'Russell_3000_Fundamentals.csv',
 'Russell_3000_With_Macro.csv',
 'X_test.csv',
 'X_test_filled.csv',
 'X_test_filled_KPIs.csv',
 'X_train.csv',
 'X_train_filled.csv',
 'X_train_filled_KPIs.csv',
 'y_test.csv',
 'y_train.csv']

In [5]:
train_file = 'X_train_filled_KPIs.csv'
test_file =  'X_test_filled_KPIs.csv'
train_dataset = pd.read_csv(train_file)
test_dataset = pd.read_csv(test_file)
print(train_dataset.head())

   Unnamed: 0 Ticker                             Name                  Sector  \
0        1564   NSSC  NAPCO SECURITY TECHNOLOGIES INC  Information Technology   
1         652   ALGN             ALIGN TECHNOLOGY INC             Health Care   
2         922   UBSI            UNITED BANKSHARES INC              Financials   
3        2094   RCUS            ARCUS BIOSCIENCES INC             Health Care   
4        1197   CRNX    CRINETICS PHARMACEUTICALS INC             Health Care   

   CapitalExpenditure_2024Q2  CapitalExpenditure_2024Q3  \
0                -551,000.00                -680,000.00   
1             -53,450,000.00             -29,800,000.00   
2              -2,973,000.00              -1,585,000.00   
3              -1,215,000.00              -1,000,000.00   
4                -955,000.00                -528,000.00   

   CapitalExpenditure_2024Q4  CapitalExpenditure_2025Q1  \
0              -1,134,000.00                 -65,000.00   
1             -22,961,000.00            

In [6]:
print(train_dataset.shape)
print(test_dataset.shape)

(1974, 147)
(496, 147)


Alright, now that we have it all loaded, we are going to have to look at quarterly changes. From the quarterly changes we can start calculating the rate of change etc. The best way to do this I think will be to design a function that will output a series that we can then just add into the dataframe. As parameters, it will take our original dataframe, the fature and the quarters to calculate the change. First, let's get a list of all the columns that we will need to calculate this for.

In [7]:
# Let's create a copy of the dataset so that we can merge data back to it later
original_train_df = train_dataset.copy()
columns = train_dataset.columns
for column in columns:
    print(column)

Unnamed: 0
Ticker
Name
Sector
CapitalExpenditure_2024Q2
CapitalExpenditure_2024Q3
CapitalExpenditure_2024Q4
CapitalExpenditure_2025Q1
CashAndSTInvestments_2024Q2
CashAndSTInvestments_2024Q3
CashAndSTInvestments_2024Q4
CashAndSTInvestments_2025Q1
CashFromOps_2024Q2
CashFromOps_2024Q3
CashFromOps_2024Q4
CashFromOps_2025Q1
CostOfRevenue_2024Q2
CostOfRevenue_2024Q3
CostOfRevenue_2024Q4
CostOfRevenue_2025Q1
CurrentAssets_2024Q2
CurrentAssets_2024Q3
CurrentAssets_2024Q4
CurrentAssets_2025Q1
CurrentLiabilities_2024Q2
CurrentLiabilities_2024Q3
CurrentLiabilities_2024Q4
CurrentLiabilities_2025Q1
EPS_2024Q2
EPS_2024Q3
EPS_2024Q4
EPS_2025Q1
Exchange
IncomeTaxExpense_2024Q2
IncomeTaxExpense_2024Q3
IncomeTaxExpense_2024Q4
IncomeTaxExpense_2025Q1
InterestExpense_2024Q2
InterestExpense_2024Q3
InterestExpense_2024Q4
InterestExpense_2025Q1
Location
LongTermDebt_2024Q2
LongTermDebt_2024Q3
LongTermDebt_2024Q4
LongTermDebt_2025Q1
Market Value
NetIncome_2024Q2
NetIncome_2024Q3
NetIncome_2024Q4
NetIncome_20

Okay, so the easy part is the quarters so let's build a list of quarters

In [8]:
quarters = ['_2024Q2','_2024Q3','_2024Q4','_2025Q1']

Now, we need to create a list of unique columns. We can do this by iterating trhough splitting adding to a set and then creating a list.

In [9]:
unique_columns = set()
for column in columns:
    column = column.replace(' ','_')
    words = column.split('_')
    if words[0] != 'KPI':
        unique_columns.add(words[0])
    else:
        unique_columns.add(str(words[0]) + '_' + str(words[1]))
unique_columns = list(unique_columns)
for column in unique_columns:
    print(column)


EPS
TotalAssets
KPI_WorkingCapital
TotalLiabilities
KPI_CurrentRatio
GDP
CashFromOps
Exchange
Name
Sector
KPI_ReturnOnEquity
Unnamed:
Inflation
CashAndSTInvestments
InterestExpense
CurrentAssets
IncomeTaxExpense
CostOfRevenue
KPI_TotalAssetTurnover
KPI_GrossProfitMargin
KPI_Leverage
TotalEquity
KPI_DebtToEquityRatio
KPI_CashFlow
Market
OperatingIncome
NetIncome
Revenue
Ticker
KPI_ReturnOnAssets
OtherOperatingExpense
KPI_NetProfitMargin
CapitalExpenditure
CurrentLiabilities
TotalDebt
Unemployment
IndustrialProd
InterestRate
Location
GDPReal
LongTermDebt


Great, now we can drop all of the features that are not quarterly as we won't be calculating the differences for these.

In [11]:
drop = {'Unnamed:','Name','Ticker','Exchange','Sector','Market','Location','Inflation','Unemployment','IndustrialProd','GDP','GDPReal','InterestRate'}

unique_columns = [c for c in unique_columns if c not in drop]
for c in unique_columns:
    print(c)

EPS
TotalAssets
KPI_WorkingCapital
TotalLiabilities
KPI_CurrentRatio
CashFromOps
KPI_ReturnOnEquity
CashAndSTInvestments
InterestExpense
CurrentAssets
IncomeTaxExpense
CostOfRevenue
KPI_TotalAssetTurnover
KPI_GrossProfitMargin
KPI_Leverage
TotalEquity
KPI_DebtToEquityRatio
KPI_CashFlow
OperatingIncome
NetIncome
Revenue
KPI_ReturnOnAssets
OtherOperatingExpense
KPI_NetProfitMargin
CapitalExpenditure
CurrentLiabilities
TotalDebt
LongTermDebt


Okay, Now we have a solid list of the raw data that we want to calcualte quarterly information for, we can start to build the function that will accomplish what we want it to. We can return a series, or we can just have the function build it into the dataset without returning anything, which is likely way more efficient. Thinking about the nesting of this, we will iterate through each unique column, we will then iterate through the quarters to calcuate the values and add it to the dataframe before moving on to the next unique column.

In [13]:
def quarterly_changes(dataset, unique_columns, quarters):
    for column in unique_columns:
        try:
            for val in range(1,len(quarters)):
                series_1 = dataset[str(column) + str(quarters[val-1])]
                series_2 = dataset[str(column) + str(quarters[val])]
                dataset[f'{column}_QoQ_{quarters[val-1][-4:]}_{quarters[val][-4:]}'] = (series_2 - series_1)/series_1
        except:
            for val in range(2,len(quarters)):
                series_1 = dataset[str(column) + str(quarters[val-1])]
                series_2 = dataset[str(column) + str(quarters[val])]
                dataset[f'{column}_QoQ_{quarters[val-1][-4:]}_{quarters[val][-4:]}'] = (series_2 - series_1)/series_1

    print(f"Completed Quarterly Change Calculations")
    return dataset


In [14]:
# Run the function to see the output
train_dataset = quarterly_changes(train_dataset, unique_columns, quarters)
print(train_dataset.shape)
train_dataset.head()

Completed Quarterly Change Calculations
(1974, 228)


,Unnamed: 0,Ticker,Name,Sector,CapitalExpenditure_2024Q2,CapitalExpenditure_2024Q3,CapitalExpenditure_2024Q4,CapitalExpenditure_2025Q1,CashAndSTInvestments_2024Q2,CashAndSTInvestments_2024Q3,...,CapitalExpenditure_QoQ_24Q4_25Q1,CurrentLiabilities_QoQ_24Q2_24Q3,CurrentLiabilities_QoQ_24Q3_24Q4,CurrentLiabilities_QoQ_24Q4_25Q1,TotalDebt_QoQ_24Q2_24Q3,TotalDebt_QoQ_24Q3_24Q4,TotalDebt_QoQ_24Q4_25Q1,LongTermDebt_QoQ_24Q2_24Q3,LongTermDebt_QoQ_24Q3_24Q4,LongTermDebt_QoQ_24Q4_25Q1
0,1564,NSSC,NAPCO SECURITY TECHNOLOGIES INC,Information Technology,"-551,000.00","-680,000.00","-1,134,000.00","-65,000.00","65,341,000.00","85,596,000.00",...,-0.94,0.11,-0.12,0.04,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01
1,652,ALGN,ALIGN TECHNOLOGY INC,Health Care,"-53,450,000.00","-29,800,000.00","-22,961,000.00","-25,289,000.00","761,429,000.00","1,041,935,000.00",...,0.10,0.01,-0.01,-0.02,-0.01,-0.07,-0.01,-0.02,-0.09,-0.02
2,922,UBSI,UNITED BANKSHARES INC,Financials,"-2,973,000.00","-1,585,000.00","-2,798,000.00","-3,895,000.00","1,857,653,000.00","1,907,579,000.00",...,0.39,-0.14,-0.08,0.19,-0.60,-0.00,0.02,-0.64,0.00,0.02
3,2094,RCUS,ARCUS BIOSCIENCES INC,Health Care,"-1,215,000.00","-1,000,000.00","-1,000,000.00","-1,000,000.00","156,000,000.00","201,000,000.00",...,-0.00,0.08,0.05,-0.15,4.27,0.03,0.00,0.46,0.02,0.00
4,1197,CRNX,CRINETICS PHARMACEUTICALS INC,Health Care,"-955,000.00","-528,000.00","-1,029,000.00","-1,239,000.00","302,162,000.00","317,269,000.00",...,0.20,-0.01,0.11,-0.04,-0.01,-0.02,-0.02,-0.02,-0.01,-0.01


In [15]:
# Need to do the same with the test
test_dataset = quarterly_changes(test_dataset, unique_columns, quarters)
print(test_dataset.shape)
test_dataset.head()

Completed Quarterly Change Calculations
(496, 228)


,Unnamed: 0,Ticker,Name,Sector,CapitalExpenditure_2024Q2,CapitalExpenditure_2024Q3,CapitalExpenditure_2024Q4,CapitalExpenditure_2025Q1,CashAndSTInvestments_2024Q2,CashAndSTInvestments_2024Q3,...,CapitalExpenditure_QoQ_24Q4_25Q1,CurrentLiabilities_QoQ_24Q2_24Q3,CurrentLiabilities_QoQ_24Q3_24Q4,CurrentLiabilities_QoQ_24Q4_25Q1,TotalDebt_QoQ_24Q2_24Q3,TotalDebt_QoQ_24Q3_24Q4,TotalDebt_QoQ_24Q4_25Q1,LongTermDebt_QoQ_24Q2_24Q3,LongTermDebt_QoQ_24Q3_24Q4,LongTermDebt_QoQ_24Q4_25Q1
0,48,TMO,THERMO FISHER SCIENTIFIC INC,Health Care,"-301,000,000.00","-272,000,000.00","-480,000,000.00","-362,000,000.00","7,073,000,000.00","4,645,000,000.00",...,-0.25,-0.01,-0.09,-0.01,-0.00,-0.11,0.09,0.03,-0.07,0.08
1,2180,SD,SANDRIDGE ENERGY INC,Energy,"-2,445,000.00","-9,986,000.00","-12,832,000.00","-6,736,000.00","209,908,000.00","92,697,000.00",...,-0.48,0.27,0.03,0.02,-0.10,0.05,-0.04,-0.13,-0.00,-0.03
2,1180,KAR,OPENLANE INC,Industrials,"-13,000,000.00","-13,100,000.00","-14,000,000.00","-11,900,000.00","60,900,000.00","132,100,000.00",...,-0.15,0.00,-0.01,0.06,-0.02,-0.15,0.00,-0.04,0.06,0.09
3,1692,APPN,APPIAN CORP CLASS A,Information Technology,"-734,000.00","-355,000.00","-511,000.00","-651,000.00","120,787,000.00","99,193,000.00",...,0.27,0.02,0.19,-0.07,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01
4,456,CSL,CARLISLE COMPANIES INC,Industrials,"-24,900,000.00","-19,300,000.00","-36,600,000.00","-29,000,000.00","1,736,300,000.00","1,530,600,000.00",...,-0.21,-0.02,-0.38,-0.11,0.00,-0.13,-0.05,0.00,0.00,0.00


Let's now take all of these rates of change, plot them and take the line of best fit so that we can get the slope as an accurate rate of change over the past 5 quarters.

In [16]:
# Let's get all of the columns again
columns = train_dataset.columns
# Use sets to avoid duplicates
QoQ_columns = set()
QoQ_quarters = set()
for column in columns:
    if 'QoQ' in column:
        # Pull out the common Column Name
        QoQ_columns.add(column[:-10])
        # Let's also pull out the QoQ
        QoQ_quarters.add(column[-10:])
QoQ_columns = list(QoQ_columns)
QoQ_quarters = list(QoQ_quarters)
# Order will be important so let's sort them
QoQ_quarters.sort()
QoQ_quarters


['_24Q2_24Q3', '_24Q3_24Q4', '_24Q4_25Q1']

Now that we have a list of all of the features, we need to start pulling out all of the data in a funciton. This is no longer functioning because of either NaN's so we need to setup a save way to do this.

In [19]:
def safe_slope(vals: list[float]) -> float:
    a = np.asarray(vals, float)
    a = a[np.isfinite(a)]
    n = a.size
    if n < 2:
        return np.nan
    t = np.arange(n, dtype=float)
    
    # OLS slope without lstsq
    St, Sy = t.sum(), a.sum()
    Stt, Sty = (t*t).sum(), (t*a).sum()
    denom = n*Stt - St*St
    if np.isclose(denom, 0.0):
        return 0.0
    
    return (n*Sty - St*Sy) / denom

def get_rate_data(row, cols, quarters):
    for col in cols:
        vals = [row.get(f"{col}{q}", np.nan) for q in quarters]
        row[f"{col}_Rate"] = safe_slope(vals)
    return row

Alright now that we have the function let's apply it to our dataset.

In [23]:
train_dataset = train_dataset.apply(lambda r: get_rate_data(r, QoQ_columns, QoQ_quarters), axis=1)
train_dataset = train_dataset.apply(lambda r: get_rate_data(r, unique_columns, quarters), axis=1)
test_dataset = test_dataset.apply(lambda r: get_rate_data(r, QoQ_columns, QoQ_quarters), axis=1)
test_dataset = test_dataset.apply(lambda r: get_rate_data(r, unique_columns, quarters), axis=1)
train_dataset.head()

,Unnamed: 0,Ticker,Name,Sector,CapitalExpenditure_2024Q2,CapitalExpenditure_2024Q3,CapitalExpenditure_2024Q4,CapitalExpenditure_2025Q1,CashAndSTInvestments_2024Q2,CashAndSTInvestments_2024Q3,...,OperatingIncome_Rate,NetIncome_Rate,Revenue_Rate,KPI_ReturnOnAssets_Rate,OtherOperatingExpense_Rate,KPI_NetProfitMargin_Rate,CapitalExpenditure_Rate,CurrentLiabilities_Rate,TotalDebt_Rate,LongTermDebt_Rate
0,1564,NSSC,NAPCO SECURITY TECHNOLOGIES INC,Information Technology,"-551,000.00","-680,000.00","-1,134,000.00","-65,000.00","65,341,000.00","85,596,000.00",...,"-913,900.00","-1,095,400.00","-2,017,700.00",-0.00,"85,800.00",-0.01,"100,400.00","-172,000.00","-45,100.00","-45,100.00"
1,652,ALGN,ALIGN TECHNOLOGY INC,Health Care,"-53,450,000.00","-29,800,000.00","-22,961,000.00","-25,289,000.00","761,429,000.00","1,041,935,000.00",...,"-11,395,700.00","-2,215,800.00","-13,033,700.00",-0.00,"156,400.00",-0.00,"9,132,200.00","-17,309,300.00","-3,999,400.00","-4,366,300.00"
2,922,UBSI,UNITED BANKSHARES INC,Financials,"-2,973,000.00","-1,585,000.00","-2,798,000.00","-3,895,000.00","1,857,653,000.00","1,907,579,000.00",...,"1,084,650.00","-3,746,200.00","10,405,300.00",-0.00,"-2,367,200.00",-0.03,"-397,900.00","-7,546,050.00","-281,094,800.00","-281,709,400.00"
3,2094,RCUS,ARCUS BIOSCIENCES INC,Health Care,"-1,215,000.00","-1,000,000.00","-1,000,000.00","-1,000,000.00","156,000,000.00","201,000,000.00",...,"-4,600,000.00","-5,900,000.00","-5,500,000.00",-0.01,"-900,000.00",-0.65,"64,500.00","-1,000,000.00","14,900,000.00","4,811,200.00"
4,1197,CRNX,CRINETICS PHARMACEUTICALS INC,Health Care,"-955,000.00","-528,000.00","-1,029,000.00","-1,239,000.00","302,162,000.00","317,269,000.00",...,"-9,281,400.00","-7,192,400.00","-11,400.00",0.01,"9,270,000.00",-82.47,"-135,300.00","1,492,400.00","-831,400.00","-636,700.00"


In [24]:
missing = train_dataset['CapitalExpenditure_QoQ_24Q4_25Q1'].isna().sum()
total = train_dataset.shape[0]
ratio =(missing/total)*100
print(f'The percent of missing data in the rates is {np.round(ratio,2)}%')

The percent of missing data in the rates is 0.15%


In [25]:
print(train_dataset.KPI_GrossProfitMargin_Rate.min())
print(train_dataset.KPI_GrossProfitMargin_Rate.max())

-9.024311153512485
94.65243828065637


## Final Section to Save new CSV

In [ ]:
# Uncomment this block when we want to save to a CSV file
train_dataset.to_csv("X_train_filled_KPIs_QoQ.csv", index=False)
test_dataset.to_csv("X_test_filled_KPIs_QoQ.csv", index=False)